In [1]:
import sys, os
import time, datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mysql.connector as cnt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns


plt.style.use(style='default')
#print(plt.style.available)
#bmh, classic, default, fivethirtyeight, ggplot ...
plt.style.use(style='bmh')


from bs4 import BeautifulSoup
from selenium import webdriver # naršyklės kontroleris
from selenium.webdriver.chrome.options import Options #Naršyklės  parametrai


import sqlite3
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import json, re
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.mouse_button import MouseButton
from datetime import datetime as dt

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
DB = sqlite3.connect('Baigiamasis_Darbas_AutoPlius.db')
C = DB.cursor()

In [3]:
print(os.getcwd())

e:\Mokslai


In [12]:
SQL = '''create table if not exists AutoPlius170
(
    id integer primary key autoincrement,
    Kuro_tipas text,
    Marke text,
    Pavaru_deze integer,
    Variklis text,
    Metai integer,
    Rida integer,
    Kaina integer,
    Defektai text,
    Tipas text,
    Spalva text,
    Technine_apziura integer,
    Varantieji_ratai text,
    Durų_skaičius integer,
    CO₂_emisija integer,
    Skelbimo_ID integer,
    VIN integer
    
)   
'''

C.execute(SQL)
DB.commit()

In [ ]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [13]:
opcijos = Options()
driver = uc.Chrome(options=opcijos)
#driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
#driver = uc.Chrome(browser_executable_path=r"C:\Program Files\Google\Chrome Beta\Application\chrome.exe")

base_url = "https://m.autoplius.lt/skelbimai/naudoti-automobiliai"


all_data = []
seen_first = set()
page = 1

def click_all_rodyti(driver):
    try:
        driver.execute_script("""
            Array.from(document.querySelectorAll('span'))
              .filter(e => e.textContent && e.textContent.trim() === 'Rodyti')
              .forEach(e => e.click());
        """)
    except:
        pass


def get_vin(driver, max_depth=7, depth=0):
    if depth > max_depth:
        return None

    
    if "Kėbulo numeris" not in driver.page_source and "VIN" not in driver.page_source:
        return None

    
    try:
        import re
        m = re.search(r"\b[A-HJ-NPR-Z0-9]{17}\b", driver.page_source)
        if m:
            return m.group(0)
    except:
        pass

    
    for fr in driver.find_elements(By.TAG_NAME, "iframe"):
        try:
            driver.switch_to.frame(fr)
            vin = get_vin(driver, max_depth, depth + 1)
            if vin:
                return vin
        except:
            pass
        finally:
            driver.switch_to.parent_frame()

    return None


while True:
    driver.get(f"{base_url}?page_nr={page}")
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    skelbimai = soup.select("a.item-thumb.js-announcement-list-item")

    if not skelbimai:
        break

    first = skelbimai[0].get("data-id") or skelbimai[0].get("href")
    if first in seen_first:
        break
    seen_first.add(first)

    for sk in skelbimai:
        href = sk.get("href")
        if not href:
            continue

        driver.get(href)
        time.sleep(2)

        driver.switch_to.default_content()
        click_all_rodyti(driver)
        time.sleep(0.7)

        driver.switch_to.default_content()
        vin = get_vin(driver)

        

        
        soup_d = BeautifulSoup(driver.page_source, "html.parser")

        marke = soup_d.select_one(".title-container")
        kaina = soup_d.select_one(".main-price")
        #print(kaina)
        metai = next(
        (t.parent.get_text(" ", strip=True).replace("Pirma registracija:", "").strip()
        for t in soup_d.select("span.view-field-title")
            if "Pirma registracija" in t.get_text(strip=True)),
        None
        )
        #print(metai)

        

        kuras = next(
        (t.parent.get_text(" ", strip=True).replace("Kuro tipas:", "").strip()
        for t in soup_d.select("span.view-field-title")
            if "Kuro tipas" in t.get_text(strip=True)),
        None
        )
        #print(kuras)
        variklis = next(
        (t.parent.get_text(" ", strip=True).replace("Variklis:", "").strip()
        for t in soup_d.select("span.view-field-title")
            if "Variklis" in t.get_text(strip=True)),
        None
        )
        #print(variklis)
        deze = next(
        (t.parent.get_text(" ", strip=True).replace("Pavarų dėžė:", "").strip()
        for t in soup_d.select("span.view-field-title")
            if "Pavarų dėžė" in t.get_text(strip=True)),
        None
        )
        #print(deze)
        rida = next((t.parent.get_text(" ", strip=True).replace("Rida:", "").strip()
        for t in soup_d.select("span.view-field-title")
            if "Rida" in t.get_text(strip=True)),
        None
        )
        #print(rida)

        spalva = next((t.parent.get_text(" ", strip=True).replace("Spalva:", "").strip()
        for t in soup_d.select("span.view-field-title")
            if "Spalva" in t.get_text(strip=True)),
        None
        )

        technine = next((t.parent.get_text(" ", strip=True).replace("Tech. apžiūra iki:", "").strip()
        for t in soup_d.select("span.view-field-title")
            if "Tech. apžiūra iki" in t.get_text(strip=True)),
        None
        )

        Varantieji_ratai = next((t.parent.get_text(" ", strip=True).replace("Varantieji ratai:", "").strip()
        for t in soup_d.select("span.view-field-title")
            if "Varantieji ratai" in t.get_text(strip=True)),
        None
        )
        #print(Varantieji_ratai)
        Durų_skaičius = next((t.parent.get_text(" ", strip=True).replace("Durų skaičius:", "").strip()
        for t in soup_d.select("span.view-field-title")
            if "Durų skaičius" in t.get_text(strip=True)),
        None
        )
        Co2 = next((t.parent.get_text(" ", strip=True).replace("CO₂ emisija, g/km:", "").strip()
        for t in soup_d.select("span.view-field-title")
            if "CO₂ emisija" in t.get_text(strip=True)),
        None
        )
        Skelbimo_ID = next((t.parent.get_text(" ", strip=True).replace("Skelbimo ID:", "").strip()
        for t in soup_d.select("span.view-field-title")
            if "Skelbimo ID" in t.get_text(strip=True)),
        None
        )


        tipas = next((t.parent.get_text(" ", strip=True).replace("Kėbulo tipas:", "").strip()
        for t in soup_d.select("span.view-field-title")
            if "Kėbulo tipas" in t.get_text(strip=True)),
        None
        )
        Defektai = next((t.parent.get_text(" ", strip=True).replace("Defektai:", "").strip()
        for t in soup_d.select("span.view-field-title")
            if "Defektai" in t.get_text(strip=True)),
        None
        )

        all_data.append((
            vin,
            kuras,
            marke.get_text(strip=True) if marke else None,
            spalva,
            Co2,
            Skelbimo_ID,
            Durų_skaičius,
            variklis,
            Varantieji_ratai,
            deze,
            technine,
            metai,
            rida,
            kaina.get_text(strip=True) if kaina else None,
            Defektai,
            tipas,
            
            
        ))

    page += 1

driver.quit()


SQL_insert = '''
INSERT INTO AutoPlius170
(VIN, Kuro_tipas, Marke, Spalva, CO₂_emisija, Skelbimo_ID, Durų_skaičius, Variklis, Varantieji_ratai, Pavaru_deze, Technine_apziura, Metai, Rida, Kaina, Defektai, Tipas)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
'''

C.executemany(SQL_insert, all_data)
DB.commit()